# This is a description of the method used to generate midi type files, containing pitches of various types

In [2]:
pip install midiutil

  Stored in directory: C:\Users\doha1\AppData\Local\pip\Cache\wheels\70\f1\24\97bde012f64820632e1e5e2935df19dfbcf9e058b0734b57cd
Successfully built midiutil
Note: you may need to restart the kernel to use updated packages.


In [20]:
from midiutil import MIDIFile
import random, numpy as np
import os

## The first method of generating pitches is by using Voss' algorithm with a Voss constraint tree
###### This function generates a truth table which is step 1 of Voss' Algorithm. The number of pitches generated by this method will be 2^diceAmount
###### For example: diceAmount=3, then the number generated pitches will be 2^3=8

In [8]:
def DiceThrow(diceAmount):
    dice=[]
    
    #create a column depending on the formula 2^n/2^m 
    m=1
    for i in range(diceAmount):
        
        div=2**m
        changeBit=2**diceAmount/div
        count=0
        col_0=[]
        col_1=[]
        full_col=[]
        #bit change for each column
        while(count<changeBit):
            col_0.append(0)
            col_1.append(1)
            count+=1
            
        #create the column
        while(len(full_col)<2**diceAmount): 
            full_col.extend(col_0)
            full_col.extend(col_1)
        
        dice.append(full_col)
        m+=1
       
 
    return dice

In [9]:
#pass the number of columns corresponding to the number of dice involved in 
#the experiment
dice=DiceThrow(3) 
print(dice)

[[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1]]


#### The next step is determining when to throw the dice
##### Each dice is represented by a column, when there is a bit change compared  to the previous column, choose a random number between 0 and drage simulating a dice throw the function returns a Voss tree

In [10]:
def CreatePitchViolin(dice):
    
    voss_constraint_tree=[]
    drange=int(128/len(dice))#determine the number of dice sides
    
    #now create the pitch (the notes), by walking in col-row order
    for i in range(len(dice[0])):#col
        for j in range(len(dice)): #row
            
            if(i==0): #if it is the very first col toss all dice
               tempPitch=random.randint(0,drange)
               #save each pitch to represent the top of the tree
               voss_constraint_tree.append([tempPitch])
            else:
                if(dice[j][i]!=dice[j][i-1]):
                    tempPitch=random.randint(0,drange)
                    voss_constraint_tree[j].append(tempPitch)
    return voss_constraint_tree

In [13]:
tree=CreatePitchViolin(dice)
print(tree)

[[14, 7], [7, 1, 38, 35], [30, 11, 6, 24, 32, 35, 23, 9]]


#### Sum of the tree

In [14]:
#sum of tree returns an array with length equal to the bottom of the tree
def VossTreeSum(tree):
    
    temp=[]
    left_tree=[]
    right_tree=[]
    
    
    #optained the left subtree but upside down
    for i in range(len(tree)-1,0,-1): #row traversal
        
        #col traversal to find the left tree
        for j in range(int(len(tree[i])/2)): 
            temp.append(tree[i][j])
        left_tree.append(temp)
        temp=[]
        
        #col traversal to find right tree
        for j in range(int(len(tree[i])/2),len(tree[i])):
            temp.append(tree[i][j])
        right_tree.append(temp)
        temp=[]
            
    left_tree.append([tree[0][0]])
    right_tree.append([tree[0][1]])
      
    
    left=[]
    right=[]
    left_sum=0
    right_sum=0
    t=0
    i=0
    j=0
    
    #left tree dimentions=right tree dimentions
    while(t<len(left_tree[0])): #repeat computations until all leafs are visited
        
        while(i<len(left_tree) and j<len(left_tree[i])):
            
            
            left_sum+=left_tree[i][j]
            right_sum+=right_tree[i][j]
            
            i+=1
            j/=2
            j=int(j)
        
        left.append(left_sum)
        right.append(right_sum)
        
        left_sum=0
        right_sum=0
            
        t+=1
        j=t
        i=0
    return left+right

In [15]:
pitches=VossTreeSum(tree)
print(pitches)

[51, 32, 21, 39, 77, 80, 65, 51]


#### The last step is creating a midi file by passing the generated pitch values. It saves a .mid type file to the disk

In [18]:
def MIDIGenerator(pitches,file_name):
    #create a midi file object to add the pitches to

     mf=MIDIFile(1,False)
     track=0
     time=0
     channel=0
     

     mf.addTrackName(track, time, "Voss Track") #
     
    #start the track with a violin on channel 1
     mf.addProgramChange(track,0,0,40) 
    #standard tempo rate 150 BPM Considered fast
     mf.addTempo(track,time,140) 
     
    
     scale_tempo=[1,1.5,1,1,1.5,1.5]
     note_duration=[1,0.5,0.5,1,0.5,1]
     t=[]
     duration=[]
     
     counter=0
     
     for i in range(int(len(pitches)/4)):
         for j in range(len(scale_tempo)):
             t.append(counter)
             counter+=scale_tempo[j]
             duration.append(note_duration[j])
            
    #a loop that adds a note using mf.addNote(track, channel, pitch, time, duration, volume)
     for i in range(1,len(pitches)):  
         mf.addTempo(track,time,140)
         mf.addNote(track,channel,pitches[i],t[i],duration[i],100)
        
     
     #write it to disk
     with open(os.path.join('midiFiles',file_name+".mid"), 'wb') as outf:
         mf.writeFile(outf)

In [19]:
MIDIGenerator(pitches,'midi_file_with_pitches2')

#### There are many methods to convert .mid files to .wav such as using the pygame package, then using the functions provided in the package. Or by using a synthisizer such as VirtualMIDISynth with a soundfont

In [4]:
import IPython
IPython.display.Audio("midi_file_with_pitches.wav")